# Module 4 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-04-programming.ipynb
    +--module-04-programming.html
    +--world.txt
    +--test01.txt
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID.

In [ ]:
from IPython.core.display import *
from StringIO import StringIO

# add whatever else you need from the Anaconda packages

## Reinforcement Learning with Q-Learning

The world for this problem is very similar to the one from Module 1 that we solved with A\* search but this time we're going to use a different approach.

We're replacing the deterministic movement with stochastic movement. This means, when the agent moves "south" instead of always going "south", there is a probability distribution of possible successor states "south", "east", "north" and "west". Thus we may not end up in the state we planned!

There are a variety of ways to handle this problem. For example, if using A\* search, if the agent finds itself off the solution, you can simply calculate a new solution from where the agent ended up. Although this sounds like a really bad idea, it has actually been shown to work really well in Video Games that use formal Planning algorithms (which we will cover later). When these algorithms were first designed, this was unthinkable. Thank you, Moore's Law!

Another approach is to use Reinforcement Learning which covers problems where there is some kind of general uncertainty. We're going to model that uncertainty a bit unrealistically here but it'll show you how the algorithm works.

As far as RL is concerned, there are a variety of options there: model-based and model-free, Value Iteration, Q-Learning and SARSA. You are going to use Value Iteration and Q-Learning.

## The World Representation

As before, we're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
^       hills      5
~       swamp      7
x       mountains  impassible
```

When you go from a plains node to a forest node it costs 3. When you go from a forest node to a plains node, it costs 1. You can think of the grid as a big graph. Each grid cell (terrain symbol) is a node and there are edges to the north, south, east and west (except at the edges).

There are quite a few differences between A\* Search and Reinforcement Learning but one of the most salient is that A\* Search returns a plan of N steps that gets us from A to Z, for example, A->C->E->G.... Reinforcement Learning, on the other hand, returns  a *policy* that tells us the best thing to do **for every and any state.**

For example, the policy might say that the best thing to do in A is go to C. However, we might find ourselves in D instead. But the policy covers this possibility, it might say, D->E. Trying this action might land us in C and the policy will say, C->E, etc. At least with offline learning, everything will be learned in advance (in online learning, you can only learn by doing and so you may act according to a known but suboptimal policy).

Nevertheless, if you were asked for a "best case" plan from (0, 0) to (n-1, n-1), you could (and will) be able to read it off the policy because there is a best action for every state. You will be asked to provide this in your assignment.

## Reading the Map

To avoid global variables, we have a <code>read_world()</code> function that takes a filename and returns the world as `List` of `List`s. **The same coordinates reversal applies: (x, y) is world[ y][ x] as from PR01.**

In [ ]:
def read_world( filename):
    with open( filename, 'r') as f:
        world_data = [x for x in f.readlines()]
    f.closed
    world = []
    for line in world_data:
        line = line.strip()
        if line == "": continue
        world.append([x for x in line])
    return world

Next we create a dict of movement costs. Note that we've negated them this time because RL requires negative costs and positive rewards:

In [ ]:
costs = { '.': -1, '*': -3, '^': -5, '~': -7}
costs

and a list of offsets for `cardinal_moves`. You'll need to work this into your actions, A, parameter.

In [ ]:
cardinal_moves = [(0,-1), (1,0), (0,1), (-1,0)]

And now the confusing bits begin. We must program both the Q-Learning algorithm and a *simulator*. The Q-Learning algorithm doesn't know T but the simulator *must*. Essentially the *simulator* is any time you apply a move and check to see what state you actually end up in (rather than the state you planned to end up in).

The transition function your *simulation* should use, T, is 0.70 for the desired direction, and 0.10 each for the other possible directions. That is, if I select "up" then 70% of the time, I go up but 10% of the time I go left, 10% of the time I go right and 10% of the time I go down. If you're at the edge of the map, you simply bounce back to the current state.

You need to implement `q_learning()` with the following parameters:

+ world: a `List` of `List`s of terrain (this is S from S, A, T, gamma, R)
+ costs: a `Dict` of costs by terrain (this is part of R)
+ goal: A `Tuple` of (x, y) stating the goal state.
+ reward: The reward for achieving the goal state.
+ actions: a `List` of possible actions, A, as offsets.
+ gamma: the discount rate
+ alpha: the learning rate

you will return a policy: 

`{(x1, y1): action1, (x2, y2): action2, ...}`

Remember...a policy is what to do in any state for all the states. Notice how this is different that A\* search which only returns actions to take from the start to the goal. This also explains why `q_learning` doesn't take a `start` state!

You should also define a function `pretty_print_policy( cols, rows, policy)` that takes a policy and prints it out as a grid using "^" for up, "<" for left, "v" for down and ">" for right. Note that it doesn't need the `world` because the policy has a move for every state. However, you do need to know how big the grid is so you can pull the values out of the `Dict` that is returned.

```
vvvvvvv
vvvvvvv
vvvvvvv
>>>>>>v
^^^>>>v
^^^>>>v
^^^>>>G
```

(Note that that policy is completely made up and only illustrative of the desired output).

There are a lot of details that I have left up to you. For example, when do you stop? Is there a strategy for learning the policy? Watch and re-watch the lecture on Q-Learning. Ask questions. You need to implement a way to pick initial states for each iteration and you need a way to balance exploration and exploitation while learning. You may have to experiment with different gamma and alpha values. Be careful with your reward...the best reward is related to the discount rate and the approxmiate number of actions you need to reach the goal.

* If everything is otherwise the same, do you think that the path from (0,0) to the goal would be the same for both A\* Search and Q-Learning?
* What do you think if you have a map that looks like:

```
><>>^
>>>>v
>>>>v
>>>>v
>>>>G
```

has this converged? Is this a "correct" policy?

**I strongly suggest that you implement Value Iteration on your own to solve these problems.** Why? Because Value Iteration will find the policy to which Q-Learning should coverge in the limit. If you include your Value Iteration implementation and output, I will count it towards your submission grade.

Remember that you should follow the general style guidelines for this course: well-named, short, focused functions with limited indentation using Markdown documentation that explains their implementation and the AI concepts behind them.

This assignment sometimes wrecks havoc with IPython notebook, save often. Put your helper functions here, along with their documentation. There should be one Markdown cell for the documentation, followed by one Codecell for the implementation.  Additionally, you may want to start your programming in a regular text editor/IDE because RL **takes a long time to run**.

----

**helper**

In [ ]:
def helper():
    pass

-----

In [ ]:
def q_learning( world, costs, goal, reward, actions, gamma, alpha):
    pass

In [ ]:
def pretty_print_policy( rows, cols, policy):
    pass

## Small World

In [ ]:
test_world = read_world( "small.txt")

In [ ]:
goal = (5, 6)
# gamma = ?? # FILL ME IN
# alpha = ?? # FILL ME IN

test_policy = q_learning( test_world, costs, goal, reward, cardinal_moves, gamma, alpha)

In [ ]:
# cols = ?? # FILL ME IN
# rows = ?? # FILL ME IN

pretty_print_policy( cols, rows, test_policy, goal)

## Full World

In [ ]:
full_world = read_world( "world.txt")

In [ ]:
goal = (26, 26) # Lower Right Corner FILL ME IN
# gamma = ?? # FILL ME IN
# alpha = ?? # FILL ME IN

full_policy = q_learning( full_world, costs, goal, reward, cardinal_moves, gamma, alpha)

In [ ]:
# cols = ?? # FILL ME IN
# rows = ?? # FILL ME IN

pretty_print_policy( cols, rows, full_policy, goal)

-----

### Value Iteration (if submitting)

Provide implementation and output of policy.